In [ ]:
!pip install pyngrok git+https://github.com/openai/whisper.git fastapi nest_asyncio uvicorn pyngrok openai


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-gv9wull_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-gv9wull_
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from fastapi import FastAPI, UploadFile, File, Form
import nest_asyncio
import uvicorn
from pyngrok import ngrok
import openai
import whisper


In [ ]:
openai.api_key = ""
ngrok.set_auth_token("")


In [ ]:
app = FastAPI()
nest_asyncio.apply()


In [ ]:
# Final FastAPI backend (image, text, video support w/ Whisper)

from fastapi import FastAPI, UploadFile, File, Form
import nest_asyncio
import uvicorn
from pyngrok import ngrok
import openai
import whisper
import threading

# Ensure only one tunnel is open
ngrok.kill()  # closes any existing tunnels
public_url = ngrok.connect(8000)
print(f"\n🎯 Your backend is live at: {public_url}/analyze/\n")


# Whisper model
model = whisper.load_model("tiny")

def transcribe(path):
    result = model.transcribe(path)
    return result["text"]

@app.post("/analyze/")
async def analyze(
    file: UploadFile = File(...),
    platform: str = Form(...),
    audience: str = Form(...)
):
    content = await file.read()

    # Detect file type
    file_ext = file.filename.split(".")[-1].lower()
    if file_ext == "txt":
        text = content.decode("utf-8")
    elif file_ext in ["mp4", "mov", "mkv", "img"]:
        with open("temp_video.mp4", "wb") as f:
            f.write(content)
        text = transcribe("temp_video.mp4")
    elif file_ext in ["jpg", "jpeg", "png"]:
        text = "This is an image. Describe content or add OCR if needed."
    else:
        return {"error": "Unsupported file type."}

    # GPT Prompt Construction
    if platform.lower() in ["youtube shorts", "instagram", "tiktok"]:
        prompt = f"""
        You are an AI content strategist. Analyze the content below and generate:
        - 5 trendy song suggestions based off of the top 100 Trendy on Spotify for today's date
        - 5 high-performing hashtags
        - 2 engaging video captions

        Platform: {platform}
        Target Audience: {audience}
        Content:
        {text}
        """
    elif platform.lower() in ["linkedin", "twitter"]:
        prompt = f"""
        You're a social media strategist for {platform}.
        Based on the following content and audience, generate:
        - 5 relevant, wide-reaching hashtags

        Target Audience: {audience}
        Content:
        {text}
        """
    else:
        return {"error": "Unsupported platform."}

    # Query OpenAI
    gpt_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
    )
    result_text = gpt_response['choices'][0]['message']['content']

    return {
        "platform": platform,
        "audience": audience,
        "recommendations": result_text
    }

# Endpoint to expose ngrok URL
@app.get("/url")
def get_ngrok_url():
    return {"url": public_url + "/analyze/"}

# Start FastAPI server in background
threading.Thread(target=lambda: uvicorn.run(app, host="0.0.0.0", port=8000)).start()


🎯 Your backend is live at: NgrokTunnel: "https://a5804ec4889b.ngrok-free.app" -> "http://localhost:8000"/analyze/



INFO:     Started server process [413]
INFO:     Waiting for application startup.
